# Homework 7 *\[handout\]*

In [ ]:
from dataclasses import dataclass
import enum

import numpy as np
import matplotlib.pyplot as plt

# your additional imports here

In [ ]:
RNG = np.random.default_rng()

# your additional global initialization here

## Background

In this homework, you will implement an initial version of the model 

Some special considerations for this homework:

- **This is a group assignment.** Each project team only has to turn in one assignment.
- **Each problem will be graded just for completion**. This is more of a project check-in than a homework with right and wrong answers. We won't be checking all of the details of your code, and coming up with tests that your code is working correctly is part of your job. We will be looking to see that you have some of those tests (and we'll provide a few suggestions).
- **The problems are somewhat open-ended.** You'll have to make decisions yourself. Feel free to change provided code if you like (even if it's in "CODE" sections instead of "ANSWER" sections).

## §1. Implementing generic Markov chain simulations

Below is a generic interface representing the state of a continuous-time Markov chain. It has two methods:

- `transition_rate`: *total* rate of transitions out of that state.
- `sample_next`: randomly generate the state that comes after this one.

See the `QueueModel` code for an example of implementing these methods for a queueing model (customers arrive at rate `rate_arrival` and are served at rate `rate_service`).

Also find below a function `run_dtmc` that simulates the "discrete-time version" of a Markov chain from a given starting state, using only `sample_next`.

> Implement `run_ctmc` to simulate a continuous-time Markov chain from a given starting state.
>
> - The output should be of the same type as `run_dtmc`: a list of `(time, state)` pairs.
>     - But, unlike `run_dtmc`, the times should be floats, not just consecutive integers.
> - Make sure your function works by plotting at least one example usage on `QueueModel`.
>
> *Hint.* Read the docs for `numpy.Generator.exponential` to make sure you pass it the right value!

#### CODE: `Model` interface

In [ ]:
class Model:

    def transition_rate(self, state):
        """Total transition rate out of the given state"""
        raise NotImplementedError

    def sample_next(self, state):
        """Generates a random state to come after the given state"""
        raise NotImplementedError

#### CODE: simulating *discrete-time* Markov chains

In [ ]:
def run_dtmc(model: Model, state_init, n_steps=256):
    path = [state_init]
    for i in range(n_steps):
        path.append(model.sample_next(path[-1]))
    # Return a list of (time, state) pairs
    # This simulation is discrete-time, so time is an integer
    return list(enumerate(path))

#### CODE: `QueueModel` example

In [ ]:
@dataclass
class QueueModel(Model):
    """Model of a queue

    The state is the number of customers currently in the queue.

    Parameters
    ----------
    rate_arrival: float
        Rate at which customers arrive at the queue.

    rate_service: float
        Rate at which customers leave the queue when it is nonempty.
    """

    rate_arrival: float
    rate_service: float

    def transition_rate(self, state: int):
        if state == 0:
            return self.rate_arrival
        else:
            return self.rate_arrival + self.rate_service

    def sample_next(self, state: int):
        if RNG.random() < self.rate_arrival / self.transition_rate(state):
            return state + 1
        else:
            return state - 1

In [ ]:
def visualize_queue_path(path):
    plt.plot(*np.asarray(path).T)

In [ ]:
visualize_queue_path(run_dtmc(QueueModel(5, 6), 0))

## §2. Implementing initial model for the project

> Implement an initial model, following exactly the dynamics outlined in Section 2 (excluding "Additional factors to consider").
>
> Test your code at least once using `run_ctmc`. Ideally, you'd run some tests that verify your implementation is correct. (For example, if your system converges towards some steady-state behavior, if you quadruple all the transition rates, do you converge to steady-state four times faster?)
>
> Some code is provided for you, but you should feel to change any or all of it.

## §3. Implementing a more extensible model for the project

As you add more features to the model, it might help to make the state more detailed. For example, if you follow the suggestion to model seniority of employees, you need to know when each employee joined the company and/or was last promoted.

> Implement a class `ComplexWorkplaceState` that represents the state in a more detailed way. For each level, keep a list of employees with some information about each employee: their gender/identity, when they joined the company, etc.
>
> Test your code by comparing its output to that of `SimpleWorkplaceModel`. For now, aim to get the *exact same dynamics* as the simple model in the following sense: `simulation_0` and `simulation_1` in the code samples below should be equivalent.
>
> Some code is provided for you, but you should feel to change any or all of it.

Code samples with the two simulations that should be equivalent:
```
params = WorkplaceParams(...)

complex_model = ComplexWorkplaceModel(params)
complex_state_init = ComplexWorkplaceState(...)
complex_path = run_ctmc(complex_model, complex_state_init)
simulation_0 = [(t, s.to_simple_state()) for (t, s) in complex_path]

simple_model = SimpleWorkplaceModel(params)
simple_state_init = complex_state_init.to_simple_state()
simple_path = run_ctmc(simple_model, simple_state_init)
simulation_1 = simple_path
```